In [1]:
# S3 prefix
prefix = "DEMO-pytorch-mnist"

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd

In [2]:
role='arn:aws:iam::237246836525:role/service-role/AmazonSageMaker-ExecutionRole-20210413T225818'

In [3]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [4]:
#WORK_DIRECTORY = "MNIST"

#data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [5]:
from sagemaker.debugger import TensorBoardOutputConfig
#from torch.utils.tensorboard import SummaryWriter

tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path='s3://{}/output/tensorboard".format(sess.default_bucket())',
    container_local_output_path='/opt/ml/output/tensorboard'
)

In [6]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = "{}.dkr.ecr.{}.amazonaws.com/mnist-test:latest".format(account, region)

tree = sage.estimator.Estimator(
    image,
    role,
    1,
    "ml.c4.2xlarge",
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess,
    script_mode= True,
    metric_definitions=[
        {"Name": "train:loss", "Regex": "train_loss:([0-9\\.]+)"},
        {"Name": "eval:loss", "Regex": "val_loss:([0-9\\.]+)"},
        
    ],
    hyperparameters={"epochs": 2},
    enable_sagemaker_metrics=True,
    tensorboard_output_config=tensorboard_output_config
)

In [7]:
tree.fit()

2021-11-27 20:41:32 Starting - Starting the training job...
2021-11-27 20:41:56 Starting - Launching requested ML instancesProfilerReport-1638045693: InProgress
......
2021-11-27 20:42:56 Starting - Preparing the instances for training......
2021-11-27 20:44:00 Downloading - Downloading input data
2021-11-27 20:44:00 Training - Downloading the training image..............Training started
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 100 K 
1 | layer_2 | Linear | 33.0 K
2 | layer_3 | Linear | 2.6 K 
-----------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)
#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]#015Validation sanity check:  50%|█████     | 1/2 [00:00<00:00,